In [1]:
from hello.hello3 import HelloApp, NotLoggedInError, BadError
import time, datetime
import requests

In [2]:
def wait1(s):
    dend = (datetime.datetime.now() + datetime.timedelta(seconds=s)).strftime("%m/%d/%Y %I:%M:%S %p")
    tprint("Sleeping %d seconds, ending at %s" %(s,dend))
    start = time.time()
    end = start + s
    try:
        while time.time() < end:
            time.sleep(5)
    except KeyboardInterrupt:
        pass

def wait2(s):
    pass
wait = wait1

hours = 3600
minutes = 60
seconds = 1

def now():
    return datetime.datetime.now().strftime("%H:%M:%S")
    
class H():
    def __init__(self, ip):
        self.ip = ip
        self.h = HelloApp(ip)
        
    def call(self, f, *args, _message=True, **kw):
        s = "(" + ", ".join(map(repr, args)) + ", ".join("%s=%r" %(k,v) for k,v in kw.items()) + ")"
        m = "%s %s: %s%s" % (self.ip, now(), f.__name__, s)
        if _message:
            tprint(m)
        while True:
            try:
                rv = f(*args, **kw)
            except NotLoggedInError:
                self.h.login()
            except requests.exceptions.ConnectTimeout as e:
                tprint(str(e))
                tprint("Make sure you're on the right network!")
            except (BadError, IOError) as e:
                tprint(str(e))
            except Exception as e: 
                tprint(str(e))
            else:
                return rv

    def startbatch(self, name):
        if self.call(self.h.batchrunning):
            self.call(self.h.endbatch)
        return self.call(self.h.startbatch, name)

    def setdo(self, mode, n2_or_sp, o2=None):
        self.call(self.h.setdo, mode, n2_or_sp,o2)

    def doman(self, n2, o2):
        self.setdo(1, n2, o2)

    def doauto(self, sp):
        self.setdo(0, sp)

    def dooff(self):
        self.setdo(2, 0, 0)

    def phoff(self):
        self.setph(2, 0, 0)

    def setph(self, mode, co2, base=None):
        self.call(self.h.setph, mode, co2, base)

    def phauto(self, sp):
        self.setph(0, sp)

    def endbatch(self):
        self.call(self.h.endbatch)

    def agauto(self, sp):
        self.call(self.h.setag, 0, sp)

    def setmg(self, sp):
        self.call(self.h.setmg, 1, sp)
        
    def getphpv(self):
        return self.call(self.h.gpmv, _message=False)['ph']['pv']


# CO2 Ramp Up/Down + Base

In [3]:
def test_co2_up(h):
    h.startbatch("ip80 2.2")
    h.phauto(7.5)
    
    # ramp up to at least 7.45
    while h.getphpv() < 7.45:
        wait(1*minutes)
    
    # the procedure calls to wait for 3 hours for stability "at set point"
    # before proceeding. In leu of more complex logic, just wait 6 hours instead
    wait(6* hours)
    
    h.endbatch()
    
def test_co2_down(h):
    h.startbatch("ip80 2.3")
    h.phauto(7.0)
    
    # ramp up to at least 7.45
    while h.getphpv() > 7.05:
        wait(1*minutes)
    
    # the procedure calls to wait for 3 hours for stability "at set point"
    # before proceeding. In leu of more complex logic, just wait 6 hours instead
    wait(6* hours)
    
    h.endbatch()
    
def test_base(h):
    h.startbatch("ip80 2.5")
    h.doman(65, 0)
    wait(5* hours)
    h.endbatch()

In [4]:
import threading
class HThread(threading.Thread):
    def __init__(self, ip):
        self.ip = ip
        super().__init__(daemon=True)
    def run(self):
        tprint("%s %s: Connecting"%(self.ip, now()))
        h = H(self.ip)
        #tprint("%s %s: Beginning CO2 Ramp Up"%(self.ip, now()))
        #test_co2_up(h)
        #tprint("%s %s: Beginning CO2 Ramp Down"%(self.ip, now()))
        #test_co2_down(h)
        tprint("%s %s: Beginning Base Test"%(self.ip, now()))
        test_base(h)
        tprint("%s %s: Testing Completed"%(self.ip, now()))

In [7]:
class TP():
    def __init__(self):
        self.threads = set()
        self._started = False
    def add(self, *args, **kw):
        t = HThread(*args, **kw)
        if self._started:
            t.start()
        self.threads.add(t)
    def start(self):
        for t in self.threads:
            t.start()
    def join(self):
        active = self.threads.copy()
        while True:
            threads = active.copy()
            for t in threads:
                if not t.is_alive():
                    active.remove(t)
            if not active:
                break
            time.sleep(5*minutes)

In [8]:
import queue
tq = queue.Queue()
tsentinel = object()
def tprint(*args, **kw):
    tq.put((args, kw))
    
def twork():
    while True:
        args, kw = tq.get()
        if args is tsentinel:
            break
        print(*args, **kw)

In [9]:
tp = TP()
for ip in ("192.168.1.6", "192.168.1.9", "192.168.1.11"):
    tp.add(ip)
tp.start()
printer = threading.Thread(target=twork, daemon=True)
printer.start()

192.168.1.11 17:06:11: Connecting
192.168.1.11 17:06:11: Beginning Base Test
192.168.1.11 17:06:11: batchrunning()
192.168.1.6 17:06:11: Connecting
192.168.1.6 17:06:11: Beginning Base Test
192.168.1.6 17:06:11: batchrunning()
192.168.1.9 17:06:11: Connecting
192.168.1.9 17:06:11: Beginning Base Test
192.168.1.9 17:06:11: batchrunning()
192.168.1.9 17:06:12: startbatch('ip80 2.4')
192.168.1.6 17:06:12: startbatch('ip80 2.4')
192.168.1.11 17:06:13: startbatch('ip80 2.4')
192.168.1.9 17:06:13: setdo(1, 65, 0)
192.168.1.6 17:06:13: setdo(1, 65, 0)
192.168.1.11 17:06:13: setdo(1, 65, 0)
Sleeping 18000 seconds, ending at 07/26/2018 10:06:17 PM
Sleeping 18000 seconds, ending at 07/26/2018 10:06:17 PM
Sleeping 18000 seconds, ending at 07/26/2018 10:06:17 PM
192.168.1.6 22:06:18: endbatch()
192.168.1.11 22:06:18: endbatch()
192.168.1.9 22:06:19: endbatch()
192.168.1.6 22:06:20: Testing Completed
192.168.1.11 22:06:20: Testing Completed
192.168.1.9 22:06:20: Testing Completed


In [ ]:
tq.put((tsentinel, None))

In [10]:
tp.join()

### 3L Analysis

In [40]:
#h = H('192.168.1.6')
basep='C:\\users\\natha\\documents\\test\\'

def bt():
    import datetime; return datetime.datetime.now().strftime("%y%m%d")

def dl(bn):
    if isinstance(bn, str):
        b = h.call(h.h.getdatareport_bybatchname, bn)
    elif isinstance(bn, tuple):
        name, d1, d2 = bn
        b = h.call(h.h.getreport_bydate, 'process_data', d1, d2)
        bn = name
    fn = p+bn+" " + bt() + ".csv"
    with open(fn, 'wb') as f:
        f.write(b)
    return fn

In [60]:
d1 = datetime.datetime(2018, 7, 26, 12+5, 7, 1) 
d2 = d1 + datetime.timedelta(hours = 12)
args = (("IP80 2.5"), d1, d2)
ps = []
for ip in ("192.168.1.6", "192.168.1.9", "192.168.1.11"):
    h = H(ip)
    p = basep + h.h.reactorname() + "\\"
    os.makedirs(p, exist_ok=True)
    for s in ("IP80 2.2", "IP80 2.3", args):
        resp = tofilename(s) if isinstance(s, str) else tofilename(s[0])
        ps.append(resp)
        dl(s)
        if not os.path.exists(resp):
            raise ValueError(resp)

In [46]:
from officelib.xllib import *
from officelib.const import xlconst as xlc


def open_file(file):
    global xl, wb, ws, cells, cr
    try:
        xl
    except NameError:
        xl = Excel()
    wb = xl.Workbooks.Open(file)
    ws = wb.Worksheets(1)
    cells = ws.Cells
    cr = cells.Range

def elform(c, ref):
    a1 = c.GetAddress(0,0)
    a2 = ref.GetAddress(1,1)
    a3 = ref.Offset(0,2).GetAddress(1,1)
    form = "=(%s-%s)*%s"%(a1, a2, a3)
    return form
    
def mk_elapsed(c, ref, ins=True):
    
    if ins:
        c.Offset(1,2).EntireColumn.Insert()
    top = c.Offset(1,2)
    bot = c.End(xlc.xlDown).Offset(1,2)
    top.Value = elform(c, ref)
    r = cr(top, bot)
    top.AutoFill(r)
    r.NumberFormat = "0.00"
    
def chart_rng(c):
    x1 = c.Offset(1,2)
    x2 = x1.End(xlc.xlDown)
    y1 = c.Offset(1,3)
    y2 = y1.End(xlc.xlDown)
    return cr(x1, x2), cr(y1, y2)

def mkch(c,v):
    chart = CreateChart(ws, xlc.xlXYScatterLinesNoMarkers)
    mkser(chart, c,v)
    
def mkser(ch, c,v):
    x, y = chart_rng(c)
    CreateDataSeries(ch, x, y, v)
    
def fixaxes():
    for co in ws.ChartObjects():
        ax = co.Chart.Axes(xlCategory, xlPrimary)
        ax.MinimumScale = 0
        
def analf(f, save=True):
    open_file(f)
    return analyze(f, save)

In [47]:
def analyze(fn, save=True):
    c1 = cells.Find("pHPV")
    ref = c1 = c1.Offset(2,1)
    mk_elapsed(c1, ref)
    ref.Offset(0, 2).Value = "24"
    mkch(c1,"pHPV")
    ch2 = CreateChart(ws, xlc.xlXYScatterLinesNoMarkers)
    mkser(ch2, c1, "pHPV")
    
    for v in "pHCO2ActualRequest(%)", "pHBaseDutyActual(%)":
        c = cells.Find(v).Offset(2,1)
        mk_elapsed(c, ref)
        mkch(c,v)
        mkser(ch2, c, v)
    fixaxes()
    if save:
        wb.SaveAs(fn.replace('.csv','.xlsx'), FileFormat=xlc.xlOpenXMLWorkbook)
    return wb

In [48]:
def tofilename(bn):
    return p + bn + " " + bt() + ".csv"

In [73]:
files = ps.copy()
ps2 = []
for file in files: 
    parts = file.split("\\")
    parts[-1] = parts[-2] +" " +parts[-1]
    file2 = "\\".join(parts)
    os.rename(file, file2)
    ps2.append(file2)

True
True
True
True
True
True
True
True
True


In [74]:
import os
xl = Excel()
files = ps
with screen_lock(xl):
    xl.DisplayAlerts = False
    for file in ps2:
        analf(os.path.abspath(file))
# 

In [75]:
wbs = list(xl.Workbooks)

In [76]:
for wb in wbs:
    wb.Worksheets(1).ChartObjects(2).Chart.SeriesCollection(2).AxisGroup = xlc.xlSecondary
    wb.Worksheets(1).ChartObjects(2).Chart.SeriesCollection(3).AxisGroup = xlc.xlSecondary

In [85]:
for wb in wbs:
    ch = wb.Worksheets(1).ChartObjects(2).Chart
    ch.SeriesCollection(2).Name = "CO\u2082 Flow"
    ch.SeriesCollection(3).Name = "Base Flow"

In [89]:
for wb in wbs:
    for co in wb.Worksheets(1).ChartObjects():
        ch = co.Chart
        ch.Axes(xlValue).HasMajorGridlines=True

In [103]:
with screen_lock(xl):
    for i, wb in enumerate(wbs):
        ws = wb.Worksheets(1)
        ws.ChartObjects(1).Chart.HasLegend = False
        ws.ChartObjects(2).Chart.HasLegend = True
        name = wb.Name.split("IP80")[0]
        if i % 3 == 0:
            name += " 7.0 to 7.5 CO\u2082 Only"
        elif i % 3 == 1:
            name += " 7.5 to 7.0 CO\u2082 Only"
        else:
            name += " Base Stability 50% CO\u2082"
        FormatChart(ws.ChartObjects(1).Chart, None, name, "Time(hr)", "pHPV")
        FormatChart(ws.ChartObjects(2).Chart, None, name, "Time(hr)", "pHPV")
        ax = ws.ChartObjects(2).Chart.Axes(xlc.xlValue, xlc.xlSecondary)
        ax.HasTitle = True
        ax.AxisTitle.Text = "Controller Request(%)"

In [104]:
with screen_lock(xl):
    for wb in wbs:
        for co in list(wb.Worksheets(1).ChartObjects())[:2]:
            co.Chart.Location(xlc.xlLocationAsNewSheet)

In [117]:
with screen_lock(xl):
    for i, wb in enumerate(wbs):
        if i < 3:
            unit = "Unit A"
        elif i < 6:
            unit = "Unit C"
        else:
            unit = "Unit B"
        m = i % 3
        if m == 0:
            name = " 7.0 to 7.5 CO\u2082 Only"
        elif m == 1:
            name = " 7.5 to 7.0 CO\u2082 Only"
        else:
            name = " Base Stability 50% CO\u2082"
        ch = wb.Charts("Chart2")
        #print(ch.ChartTitle.Text)
        ch.ChartTitle.Text = unit + name

In [125]:
for wb in wbs:
    wb.Charts("Chart2").PrintOut()

In [29]:
def phxtime(idx, a, b):
    wb = wbs[idx]
    for co in wb.Worksheets(1).ChartObjects():
        ch = co.Chart
        FormatAxesScale(ch, a, b)
        
def phytime(idx, a, b):
    wb = wbs[idx]
    for co in wb.Worksheets(1).ChartObjects():
        ch = co.Chart
        FormatAxesScale(ch, None, None, a, b)
        
def phctype(idx, typ):
    wb = wbs[idx]
    wb.Worksheets(1).ChartObjects(1).Chart.ChartType = typ

In [80]:
for i in range(2, 9, 3):
    phytime(i, 6.94, 7.06)

In [ ]:
phytime(0, 7.45, 7.55)
ph

In [ ]:
phxtime(0, 2, 6)
phxtime(2, 2, 6)

In [ ]:
phxtime(2, 10, 14)

In [ ]:
phctype(0, xlc.xlXYScatterLines)
phctype(2, xlc.xlXYScatterLines)

In [ ]:
phxtime(1, 2,6)
phxtime(3, 10, 14)

In [ ]:
def bgetc(idx):
    return xl.Workbooks("IE21 CO2 Test 180607.xlsx").Worksheets(1).ChartObjects(idx).Chart

def bxfix(idx, a, b):
    ch = bgetc(idx)
    FormatAxesScale(ch, a, b)
    
def byfix(idx, a, b):
    ch = bgetc(idx)
    FormatAxesScale(ch, None, None, a, b)

In [ ]:
for co in xl.Workbooks(5).Worksheets(1).ChartObjects():
    ch = co.Chart
    FormatAxesScale(ch, 0, 6)